In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")

In [2]:
np.random.seed(2018)
# Load data.
trn = pd.read_csv('Train_Cleaned.csv')
tst = pd.read_csv('Test_Cleaned.csv')

In [3]:
## Data preprocessing ##

# Store product variables separately.
prods = trn.columns[22:].tolist()

In [4]:
prods

['Saving_account',
 'Guarantees',
 'Cur_account',
 'Derivative_account',
 'Payroll_account',
 'Junior_account',
 'Particular_acct1',
 'Particular_acct2',
 'Particular_acct3',
 'Short_term_deposites',
 'Med_term_deposites',
 'Long_term_deposites',
 'e-account',
 'Funds',
 'Mortgage',
 'Pension',
 'Loans',
 'Taxes',
 'Credit_card',
 'Securities',
 'Home_account',
 'Payroll',
 'Pensions',
 'Direct_debit']

In [5]:
trn.isnull().sum()

Month_status_date         0
Customer_ID               0
Employee_Index            0
Customer_country          0
Sex                       0
Age                       0
Join_date                 0
New_customer              0
Relnshp_Mnths             0
Relnshp_flag              0
Last_date_Prim_Cust       0
Cust_type_beg_Mth         0
Cust_Reln_type_beg_mth    0
Residence_flag            0
Forigner_flag             0
Emp_spouse_flag           0
Channel_when_joined       0
Deceased_flag             0
Address_detail            0
Activity_flag             0
Gross_household_income    0
Segment                   0
Saving_account            0
Guarantees                0
Cur_account               0
Derivative_account        0
Payroll_account           0
Junior_account            0
Particular_acct1          0
Particular_acct2          0
Particular_acct3          0
Short_term_deposites      0
Med_term_deposites        0
Long_term_deposites       0
e-account                 0
Funds               

In [6]:
trn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 46 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Month_status_date       object 
 1   Customer_ID             int64  
 2   Employee_Index          object 
 3   Customer_country        object 
 4   Sex                     object 
 5   Age                     int64  
 6   Join_date               object 
 7   New_customer            int64  
 8   Relnshp_Mnths           int64  
 9   Relnshp_flag            int64  
 10  Last_date_Prim_Cust     object 
 11  Cust_type_beg_Mth       object 
 12  Cust_Reln_type_beg_mth  object 
 13  Residence_flag          object 
 14  Forigner_flag           object 
 15  Emp_spouse_flag         object 
 16  Channel_when_joined     object 
 17  Deceased_flag           object 
 18  Address_detail          object 
 19  Activity_flag           int64  
 20  Gross_household_income  float64
 21  Segment                 objec

In [7]:
# Remove customer data that doesn't have any of the 24 products.
no_product = trn[prods].sum(axis=1) == 0
trn = trn[~no_product]

In [8]:
# Integrate training data and test data. Product variables that are not in the test data are filled with zeros.
for col in trn.columns[22:]:
    tst[col] = 0
df = pd.concat([trn, tst], axis=0)

In [9]:
df.drop(['Last_date_Prim_Cust','Emp_spouse_flag'], axis = 1, inplace=True)

In [10]:
# This is a list containing variables to be used for learning.
features = []

# Label-encode categorical variables through the .factorize() function.
categorical_cols = ['Employee_Index', 'Customer_country', 'Sex', 'Cust_Reln_type_beg_mth', 'Residence_flag',
                    'Forigner_flag', 'Channel_when_joined', 'Deceased_flag', 'Address_detail', 'Segment']
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_cols

In [11]:
df.loc[df.Relnshp_Mnths <0, "Relnshp_Mnths"] = 0

In [12]:
df['Cust_type_beg_Mth'].unique()

array(['1', 'P', '3', '2', '4', 1, 3, 2, 4], dtype=object)

In [13]:
df['Cust_type_beg_Mth'].replace('P', 5, inplace=True)
df['Cust_type_beg_Mth'].replace('1', 1, inplace=True)
df['Cust_type_beg_Mth'].replace('2', 2, inplace=True)
df['Cust_type_beg_Mth'].replace('3', 3, inplace=True)
df['Cust_type_beg_Mth'].replace('4', 4, inplace=True)

In [14]:
df['Cust_type_beg_Mth'].unique()

array([1, 5, 3, 2, 4])

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12020685 entries, 0 to 929614
Data columns (total 44 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Month_status_date       object 
 1   Customer_ID             int64  
 2   Employee_Index          int64  
 3   Customer_country        int64  
 4   Sex                     int64  
 5   Age                     int64  
 6   Join_date               object 
 7   New_customer            int64  
 8   Relnshp_Mnths           int64  
 9   Relnshp_flag            int64  
 10  Cust_type_beg_Mth       int64  
 11  Cust_Reln_type_beg_mth  int64  
 12  Residence_flag          int64  
 13  Forigner_flag           int64  
 14  Channel_when_joined     int64  
 15  Deceased_flag           int64  
 16  Address_detail          int64  
 17  Activity_flag           int64  
 18  Gross_household_income  float64
 19  Segment                 int64  
 20  Saving_account          int64  
 21  Guarantees              int64  

In [16]:
# Seek features for numeric variables to be used for learning.
features += ['Age','Relnshp_Mnths','Gross_household_income','New_customer','Relnshp_flag','Cust_type_beg_Mth','Activity_flag']

In [17]:
features

['Employee_Index',
 'Customer_country',
 'Sex',
 'Cust_Reln_type_beg_mth',
 'Residence_flag',
 'Forigner_flag',
 'Channel_when_joined',
 'Deceased_flag',
 'Address_detail',
 'Segment',
 'Age',
 'Relnshp_Mnths',
 'Gross_household_income',
 'New_customer',
 'Relnshp_flag',
 'Cust_type_beg_Mth',
 'Activity_flag']

In [18]:
df['Join_date'].unique()

array(['2015-01-12', '2012-08-10', '2011-09-06', ..., '2013-06-15',
       '2012-04-29', '2012-08-19'], dtype=object)

In [19]:
len(df['Join_date'].unique())

6756

In the feature engineering stage, derived variables are created to be used for machine learning model training. In the baseline model, a total of 24 customer variables, 4 date-based derived variables, and 24 lag-1 variables are used.

The year and month information are extracted from fecha_alta, which means the date when the customer signed the first contract, and ult_fec_cli_1t, which means the date when the customer was last grade 1, respectively. In addition, there are various derived variables using date variables. For example, the difference between two date variables can be created as a derived variable.

Missing values are temporarily replaced with -99. The machine learning model provided by Scikit-learn does not receive a missing value as an input value and an execution error occurs, but in the xgboost model, the missing value is also received as a normal input value. The missing data is recognized as one piece of information and used for model training, but I will set the missing value to -99.

In time series data, various derived variables can be created based on customer's past data. For example, whether a customer's age has changed in the last 3 months (i.e., has a birthday within 3 months) can be created as a binary variable, or information about a product purchased a month ago can be used as a variable, You can also calculate the average monthly salary for the last six months.

Starting from the conclusion, the lag variable indicating whether or not a financial product was held N months ago served as a good derivative. It is to use as current customer data whether it was held 1 month ago, 2 months ago, or 3 months ago for 24 financial product variables. In the baseline model, we will use the lag-1 to lag-5 variables that retrieve information from 1 month ago to 5 months ago.

In [20]:
# (Feature Engineering) Extract year and month information from two date variables.
df['Join_date_month'] = df['Join_date'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['Join_date_year'] = df['Join_date'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['Join_date_month', 'Join_date_year']

In [21]:
#All missing values of other variables are replaced with -99.
df.fillna(-99, inplace=True)

# (Feature Engineering) Generate lag-1 data.

# This is a function that converts a date to a number. 2015-01-28 is converted to 1, 2016-06-28 is converted to 18
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] 
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

# Convert the date to a number and store it in int_date
df['int_date'] = df['Month_status_date'].map(date_to_int).astype(np.int8)

In [22]:
#Convert the date to a number, copy the data # to int_date, and add 1 to the int_date date to create a lag. 
#Add _prev to variable name and save
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['Customer_ID', 'int_date'] else col for col in df.columns ]
df_lag['int_date'] += 1

In [23]:
# Here I made the change
df_trn = df.merge(df_lag, on=['Customer_ID','int_date'], how='left')

In [24]:
df.head()

,Month_status_date,Customer_ID,Employee_Index,Customer_country,Sex,Age,Join_date,New_customer,Relnshp_Mnths,Relnshp_flag,Cust_type_beg_Mth,Cust_Reln_type_beg_mth,Residence_flag,Forigner_flag,Channel_when_joined,Deceased_flag,Address_detail,Activity_flag,Gross_household_income,Segment,Saving_account,Guarantees,Cur_account,Derivative_account,Payroll_account,Junior_account,Particular_acct1,Particular_acct2,Particular_acct3,Short_term_deposites,Med_term_deposites,Long_term_deposites,e-account,Funds,Mortgage,Pension,Loans,Taxes,Credit_card,Securities,Home_account,Payroll,Pensions,Direct_debit,Join_date_month,Join_date_year,int_date
0,2015-01-28,1375586,0,0,0,35,2015-01-12,0,6,1,1,0,0,0,0,0,0,1,87218.10,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2015,1
1,2015-01-28,1050611,0,0,1,23,2012-08-10,0,35,1,1,1,0,1,1,0,1,0,35548.74,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,2012,1
2,2015-01-28,1050612,0,0,1,23,2012-08-10,0,35,1,1,1,0,0,1,0,1,0,122179.11,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,2012,1
3,2015-01-28,1050613,0,0,0,22,2012-08-10,0,35,1,1,1,0,0,2,0,2,0,119775.54,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,2012,1
4,2015-01-28,1050614,0,0,1,23,2012-08-10,0,35,1,1,0,0,0,1,0,2,1,99950.28,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,2012,1


In [25]:
# Remove unnecessary variables from memory for memory efficiency
del df, df_lag

In [26]:
# Replace with 0 in case the product information for the last month does not exist.
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)

In [27]:
df_trn.fillna(-99, inplace=True)

In [28]:

# Add the lag-1 variable.
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]
### After Baseline model, various feature engineering added

In [29]:
features

['Employee_Index',
 'Customer_country',
 'Sex',
 'Cust_Reln_type_beg_mth',
 'Residence_flag',
 'Forigner_flag',
 'Channel_when_joined',
 'Deceased_flag',
 'Address_detail',
 'Segment',
 'Age',
 'Relnshp_Mnths',
 'Gross_household_income',
 'New_customer',
 'Relnshp_flag',
 'Cust_type_beg_Mth',
 'Activity_flag',
 'Join_date_month',
 'Join_date_year',
 'Employee_Index_prev',
 'Customer_country_prev',
 'Sex_prev',
 'Cust_Reln_type_beg_mth_prev',
 'Residence_flag_prev',
 'Forigner_flag_prev',
 'Channel_when_joined_prev',
 'Deceased_flag_prev',
 'Address_detail_prev',
 'Segment_prev',
 'Age_prev',
 'Relnshp_Mnths_prev',
 'Gross_household_income_prev',
 'New_customer_prev',
 'Relnshp_flag_prev',
 'Cust_type_beg_Mth_prev',
 'Activity_flag_prev',
 'Join_date_month_prev',
 'Join_date_year_prev',
 'Saving_account_prev',
 'Guarantees_prev',
 'Cur_account_prev',
 'Derivative_account_prev',
 'Payroll_account_prev',
 'Junior_account_prev',
 'Particular_acct1_prev',
 'Particular_acct2_prev',
 'Particu

In [30]:
df_trn.head()

,Month_status_date,Customer_ID,Employee_Index,Customer_country,Sex,Age,Join_date,New_customer,Relnshp_Mnths,Relnshp_flag,Cust_type_beg_Mth,Cust_Reln_type_beg_mth,Residence_flag,Forigner_flag,Channel_when_joined,Deceased_flag,Address_detail,Activity_flag,Gross_household_income,Segment,Saving_account,Guarantees,Cur_account,Derivative_account,Payroll_account,Junior_account,Particular_acct1,Particular_acct2,Particular_acct3,Short_term_deposites,Med_term_deposites,Long_term_deposites,e-account,Funds,Mortgage,Pension,Loans,Taxes,Credit_card,Securities,Home_account,Payroll,Pensions,Direct_debit,Join_date_month,Join_date_year,int_date,Month_status_date_prev,Employee_Index_prev,Customer_country_prev,Sex_prev,Age_prev,Join_date_prev,New_customer_prev,Relnshp_Mnths_prev,Relnshp_flag_prev,Cust_type_beg_Mth_prev,Cust_Reln_type_beg_mth_prev,Residence_flag_prev,Forigner_flag_prev,Channel_when_joined_prev,Deceased_flag_prev,Address_detail_prev,Activity_flag_prev,Gross_household_income_prev,Segment_prev,Saving_account_prev,Guarantees_prev,Cur_account_prev,Derivative_account_prev,Payroll_account_prev,Junior_account_prev,Particular_acct1_prev,Particular_acct2_prev,Particular_acct3_prev,Short_term_deposites_prev,Med_term_deposites_prev,Long_term_deposites_prev,e-account_prev,Funds_prev,Mortgage_prev,Pension_prev,Loans_prev,Taxes_prev,Credit_card_prev,Securities_prev,Home_account_prev,Payroll_prev,Pensions_prev,Direct_debit_prev,Join_date_month_prev,Join_date_year_prev
0,2015-01-28,1375586,0,0,0,35,2015-01-12,0,6,1,1,0,0,0,0,0,0,1,87218.10,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2015,1,-99,-99.0,-99.0,-99.0,-99.0,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0
1,2015-01-28,1050611,0,0,1,23,2012-08-10,0,35,1,1,1,0,1,1,0,1,0,35548.74,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,2012,1,-99,-99.0,-99.0,-99.0,-99.0,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0
2,2015-01-28,1050612,0,0,1,23,2012-08-10,0,35,1,1,1,0,0,1,0,1,0,122179.11,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,2012,1,-99,-99.0,-99.0,-99.0,-99.0,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0
3,2015-01-28,1050613,0,0,0,22,2012-08-10,0,35,1,1,1,0,0,2,0,2,0,119775.54,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,2012,1,-99,-99.0,-99.0,-99.0,-99.0,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0
4,2015-01-28,1050614,0,0,1,23,2012-08-10,0,35,1,1,0,0,0,1,0,2,1,99950.28,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,2012,1,-99,-99.0,-99.0,-99.0,-99.0,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0


In [31]:
df_trn.to_csv('Final_data.csv', index=False)